In [1]:
import numpy as np
import pandas as pd

### This pipeline focus on getting LOF-voted graph-embedding score from candidates file
Steps as following:
- generate required 4 features for voting classifier: 'aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized'
- generate model-voted candidates result
- in score-using-embedding: use --column-vector-strategy==centroid-of-lof and specified --lof-strategy to obtain lof-voted candidates as centroid and compute respective graph-embedding-score

In [16]:
HOME_DIR = '/Users/summ7t/dev/novartis/table-linker/t2dv2-dev-score/pipeline'
input_candidate_file_path = f'{HOME_DIR}/temp/candidates.csv'
aligned_pagerank_candidate_file_path = f'{HOME_DIR}/apr_test.csv'
model_voted_candidate_file_path = f'{HOME_DIR}/mv_test.csv'
graph_embedding_file_path = f'{HOME_DIR}/score_test.csv'

graph_embedding_complex_file = f'{HOME_DIR}/temp/graph_embedding_complex.tsv'
index_url = 'http://ckg07:9200/wikidatadwd-augmented/'

##### Generate required 4 features for voting classifier

In [6]:
!tl align-page-rank $input_candidate_file_path \
/ string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan \
/ string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
/ normalize-scores -c des_cont_jaccard / smallest-qnode-number \
> $aligned_pagerank_candidate_file_path

In [12]:
features_df = pd.read_csv(aligned_pagerank_candidate_file_path)
features_df.loc[:, ['method', 'pagerank', 'aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized']].head()

,method,pagerank,aligned_pagerank,smallest_qnode_number,monge_elkan,des_cont_jaccard_normalized
0,exact-match,3.983031e-09,3.983031e-09,0,1.000000,0.0
1,fuzzy-augmented,3.983031e-09,0.000000e+00,0,1.000000,0.0
2,fuzzy-augmented,5.918546e-09,0.000000e+00,0,0.772222,0.0
3,fuzzy-augmented,3.740191e-09,0.000000e+00,0,0.640476,0.0
4,fuzzy-augmented,0.000000e+00,0.000000e+00,0,0.750000,0.0


##### Generate model-voted candidates result

In [11]:
!tl vote-by-classifier $aligned_pagerank_candidate_file_path \
--prob-threshold 0.995 \
--model /Users/summ7t/dev/novartis/table-linker/weighted_lr.pkl \
> $model_voted_candidate_file_path

In [13]:
model_voted_df = pd.read_csv(model_voted_candidate_file_path)
model_voted_df.head()

,column,row,label,context,label_clean,kg_id,kg_labels,kg_aliases,method,kg_descriptions,pagerank,retrieval_score,aligned_pagerank,monge_elkan,des_cont_jaccard,des_cont_jaccard_normalized,smallest_qnode_number,vote_by_classifier
0,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q213854,Virat Kohli,Cheeku,exact-match,Indian cricket player,3.983031e-09,21.693314,3.983031e-09,1.000000,0.0,0.0,0,0
1,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q213854,Virat Kohli,Cheeku,fuzzy-augmented,Indian cricket player,3.983031e-09,36.393840,0.000000e+00,1.000000,0.0,0.0,0,0
2,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q102354285,Marie Virat,NaN,fuzzy-augmented,Ph. D. 2009,5.918546e-09,23.484630,0.000000e+00,0.772222,0.0,0.0,0,0
3,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q16027751,Bernard Virat,NaN,fuzzy-augmented,French biologist (1921-2003),3.740191e-09,23.484630,0.000000e+00,0.640476,0.0,0.0,0,0
4,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q7907059,VIRAT,NaN,fuzzy-augmented,NaN,0.000000e+00,20.582134,0.000000e+00,0.750000,0.0,0.0,0,0


##### Generate graph-embedding-score using centroid-of-lof and lof-strategy

In [17]:
!tl score-using-embedding $model_voted_candidate_file_path \
--column-vector-strategy centroid-of-lof \
--lof-strategy ems-mv \
-o graph-embedding-score \
--embedding-file $graph_embedding_complex_file \
--embedding-url $index_url \
> $graph_embedding_file_path

Qnodes to lookup: 1260
Qnodes from file: 1241
Qnodes from server: 0
_centroid_of_lof: Missing 1 of 18


In [19]:
score_df = pd.read_csv(graph_embedding_file_path)
score_df.head(10)

,column,row,label,context,label_clean,kg_id,kg_labels,kg_aliases,method,kg_descriptions,pagerank,retrieval_score,aligned_pagerank,monge_elkan,des_cont_jaccard,des_cont_jaccard_normalized,smallest_qnode_number,vote_by_classifier,graph-embedding-score
0,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q213854,Virat Kohli,Cheeku,exact-match,Indian cricket player,3.983031e-09,21.693314,3.983031e-09,1.000000,0.0,0.0,0,0,0.841910
1,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q213854,Virat Kohli,Cheeku,fuzzy-augmented,Indian cricket player,3.983031e-09,36.393840,0.000000e+00,1.000000,0.0,0.0,0,0,0.841910
2,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q102354285,Marie Virat,NaN,fuzzy-augmented,Ph. D. 2009,5.918546e-09,23.484630,0.000000e+00,0.772222,0.0,0.0,0,0,0.455871
3,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q16027751,Bernard Virat,NaN,fuzzy-augmented,French biologist (1921-2003),3.740191e-09,23.484630,0.000000e+00,0.640476,0.0,0.0,0,0,0.587621
4,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q7907059,VIRAT,NaN,fuzzy-augmented,NaN,0.000000e+00,20.582134,0.000000e+00,0.750000,0.0,0.0,0,0,0.000000
5,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q2978459,Virata,Virat,fuzzy-augmented,character from the epic Mahabharata,6.890132e-09,20.520416,0.000000e+00,0.725000,0.0,0.0,0,0,0.508063
6,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q16682735,NaN,NaN,fuzzy-augmented,NaN,3.539613e-09,19.623405,0.000000e+00,0.000000,0.0,0.0,0,0,0.180011
7,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q6426050,Kohli,NaN,fuzzy-augmented,NaN,3.539613e-09,19.601744,0.000000e+00,0.750000,0.0,0.0,0,0,0.327577
8,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q46251,Fränzi Mägert-Kohli,Franziska Kohli|Fraenzi Maegert-Kohli,fuzzy-augmented,Swiss snowboarder,3.539613e-09,19.233713,0.000000e+00,0.488889,0.0,0.0,1,0,0.472100
9,0,0,Virat Kohli,royal challengers bangalore|152|5/11/88,Virat Kohli,Q16434086,Wirat Wachirarattanawong,NaN,fuzzy-augmented,NaN,3.539613e-09,19.010628,0.000000e+00,0.685185,0.0,0.0,0,0,0.544198


In [20]:
score_df.sort_values(by=['graph-embedding-score'], ascending=False).loc[:, ['kg_id', 'kg_labels', 'kg_descriptions', 'method', 'vote_by_classifier', 'graph-embedding-score']].head(20)

,kg_id,kg_labels,kg_descriptions,method,vote_by_classifier,graph-embedding-score
666,Q137669,Ajinkya Rahane,Indian cricketer,exact-match,1,0.960900
667,Q137669,Ajinkya Rahane,Indian cricketer,fuzzy-augmented,1,0.960900
242,Q3522062,Ishant Sharma,Indian cricket player.,fuzzy-augmented,0,0.955415
241,Q3522062,Ishant Sharma,Indian cricket player.,exact-match,0,0.955415
140,Q142613,Cheteshwar Pujara,Indian cricket player,exact-match,1,0.950391
141,Q142613,Cheteshwar Pujara,Indian cricket player,fuzzy-augmented,1,0.950391
891,Q2003153,Bhuvneshwar Kumar,Indian cricket player,fuzzy-augmented,0,0.942442
890,Q2003153,Bhuvneshwar Kumar,Indian cricket player,exact-match,0,0.942442
1235,Q7336038,Rishi Dhawan,Indian cricketer,fuzzy-augmented,0,0.940343
578,Q16227998,Jasprit Bumrah,cricketer,exact-match,0,0.940094
